In [ ]:
import os
from os.path import join as pjoin

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold

RS = 42
SHUFFLE = True

In [ ]:
train = pd.read_csv(
    "/gpfs/space/projects/BetterMedicine/volodymyr1/exps/bird_clef_2025/birdclef_2025/train_and_prev_comps_extendedv1_pruneSL_XConly2025_snipet28032025.csv"
)

In [ ]:
def stratified_k_fold(X, y, g, k):
    return StratifiedGroupKFold(n_splits=k, shuffle=SHUFFLE, random_state=RS).split(
        X, y, g
    )

In [ ]:
train["group"] = train["author"].str.lower()
train.loc[train["group"] == "unknown", "group"] = train.loc[train["group"] == "unknown", "filename"]

In [ ]:
split = list(stratified_k_fold(X=train, y=train["primary_label"], g=train["group"], k=5))

In [ ]:
all_unique_birds = set(train["primary_label"])
len(all_unique_birds)

In [ ]:
split = [list(el) for el in split]
for fold_i in range(len(split)):
    fold_train = train.iloc[split[fold_i][0]]
    missing_birds = all_unique_birds - set(fold_train["primary_label"])
    if missing_birds:
        print(f"Fold {fold_i} missing species: {missing_birds}")
        index_to_add = train.loc[train["primary_label"].isin(missing_birds)].sample(frac=1).drop_duplicates("primary_label").index.to_list()
        print(f"Adding next filenames:\n", train["filename"].iloc[index_to_add])
        split[fold_i][0] = np.concatenate([split[fold_i][0], np.array(index_to_add)])
split = [tuple(el) for el in split]      

In [ ]:
for train_idx, _ in split:
    assert set(train["primary_label"].iloc[split[fold_i][0]]) == all_unique_birds

In [ ]:
assert list(train.index) == list(range(len(train)))

In [ ]:
train.loc[train["primary_label"].isin(missing_birds)].sample(frac=1).drop_duplicates("primary_label").index.to_list()

In [ ]:
train["group"]

In [ ]:
train["group"].str.lower().value_counts().to_dict()